In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## CHINOOK RECORDS DATABASE - A BUSINESS ANALYSIS REPORT ##

The record store is signing a deal with a new record label and we are tasked with answering some important questions that will help the business get answers to make good decisions to acquire most profit.

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


 ### Genres that sell the best in the USA among the prospective new artists that make music in - Hip-Hop, Punk, Pop, Blues ###

In [3]:
%%sql
WITH track_genre AS
(
  SELECT i.quantity,
         i.track_id,
         g.name AS genre_name,
         t.album_id
    FROM invoice_line i
    INNER JOIN track t ON t.track_id = i.track_id
    INNER JOIN genre g ON g.genre_id = t.genre_id
    GROUP BY i.track_id
)
SELECT genre_name,
       SUM(quantity) total_sold 
       FROM track_genre 
        wHERE genre_name like "%Hip Hop%" or 
        genre_name like "%Punk%" or 
        genre_name = "Pop" or
        genre_name = "Blues"
        GROUP BY genre_name
        ORDER BY total_sold DESC
        ;
       

Done.


genre_name,total_sold
Alternative & Punk,176
Blues,56
Pop,25
Hip Hop/Rap,21


From the list of provided artists and their genres, chinook records should purchase music from: 
-  **Red Tone (Punk) is our top choice**, 
-  **SLIM JIM BITES, a Blues artist** 
-  **Meteor and the girls, pop artists**

In [4]:
%%sql
WITH support_sales AS
(
SELECT  CAST(SUM(i.total) AS INTEGER) total_sales_amount,
        MIN(i.total) highest_val,
        c.support_rep_id,
        e.first_name || " " || e.last_name sales_name
        FROM invoice i
    LEFT JOIN customer c ON c.customer_id = i.customer_id
    LEFT JOIN employee e ON e.employee_id = c.support_rep_id
    GROUP BY support_rep_id
)
SELECT sales_name, total_sales_amount, support_rep_id
FROM support_sales
ORDER BY total_sales_amount DESC;

Done.


sales_name,total_sales_amount,support_rep_id
Jane Peacock,1731,3
Margaret Park,1584,4
Steve Johnson,1393,5


Jane Peacock has the highest sales record amounting to 1731 dollars. The least sales record is held by Steve Johnson.

In [5]:
%%sql
SELECT *

FROM employee
WHERE employee_id = 3 or
employee_id = 4 or
employee_id = 5
;


Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


A quick look-up at the employee table tells us that the least performing salesman with **Steve Johnson has a phone number with area code in Alberta, Calgary** whereas the **other two agents have a number from the Calgary area** which is where the office is. Steve Johnson seems to be **working remotely** which could explain his lower sales numbers.

### Analyzing sales data from different countries ###

We are going to look at sales by country and average order value by country to determine:
- Country with the most overall sales.
- Country with the highest average order value.

Some countries have only 1 customer, we will label those countries as other.

In [6]:
%%sql
DROP VIEW IF EXISTS cust_count;
CREATE VIEW cust_count AS

SELECT 
COUNT(customer_id) ct,
country,

CASE 
WHEN COUNT(customer_id)= 1 THEN 'other'
END AS sort

FROM customer
GROUP BY country
ORDER BY sort;

 

SELECT *

FROM cust_count

;

Done.
Done.
Done.


ct,country,sort
5,Brazil,None
8,Canada,None
2,Czech Republic,None
5,France,None
4,Germany,None
2,India,None
2,Portugal,None
13,USA,None
3,United Kingdom,None
1,Argentina,other


In [7]:
%%sql
DROP VIEW IF EXISTS avg_order_val;
CREATE VIEW avg_order_val AS

SELECT 
country,
avg(total) av_ord_val
FROM customer c
INNER JOIN 
(
    SELECT invoice_id,
    total,
    customer_id ci
    FROM invoice i
    GROUP BY invoice_id)  ON ci = c.customer_id
GROUP BY country;


Done.
Done.


[]

In [8]:
%%sql
SELECT c.country, 
ct customer_count,
SUM(tot) total_sales,
AVG(tots) avg_sales_customer,
av.av_ord_val,
cc.sort
FROM customer c
INNER JOIN (
    SELECT SUM(i.total) tot,
    i.total tots,
    customer_id id_c
    FROM invoice i
    GROUP BY customer_id )
ON id_c = c.customer_id
INNER JOIN cust_count cc ON cc.country = c.country
INNER JOIN avg_order_val av ON av.country = c.country
GROUP BY c.country
ORDER BY cc.sort, total_sales DESC;


Done.


country,customer_count,total_sales,avg_sales_customer,av_ord_val,sort
USA,13,1040.49,8.376923076923076,7.942671755725189,None
Canada,8,535.59,7.30125,7.047236842105265,None
Brazil,5,427.67999999999995,9.108,7.0111475409836075,None
France,5,389.07,8.91,7.781399999999998,None
Germany,4,334.62,7.672499999999999,8.161463414634147,None
Czech Republic,2,273.24,5.94,9.108000000000002,None
United Kingdom,3,245.51999999999998,6.6000000000000005,8.768571428571429,None
Portugal,2,185.13,4.455,6.383793103448276,None
India,2,183.14999999999998,3.96,8.72142857142857,None
Ireland,1,114.83999999999997,17.82,8.833846153846151,other


**According our findings**
- USA has the highest total sales amounting to USD 1040 with 13 customers
- Czech Republic has the highest average order value at USD 9.1 


### Calculating number of invoices with album sales ###

The company wants to know if customer tend to purchase whole albums or just a few tracks from an album. To answer that question we take a look at the invoice_line and track tables. We will calculate:

- Number of invoices
- total number of invoices with album purchases

In [89]:
%%sql
WITH invoice_album AS

(
    SELECT il.invoice_id, 
         il.track_id, 
         t.album_id 
FROM invoice_line il
INNER JOIN (SELECT * FROM track) t ON t.track_id = il.track_id
),

tracks_count AS
(
Select invoice_id,
       ia.album_id,
       COUNT(ia.album_id) tracks_invoice,
        tracks_album
    FROM invoice_album ia
    LEFT JOIN 
    ( SELECT COUNT(t.track_id) tracks_album, album_id AS tai
     FROM track t
     GROUP BY tai) ON tai= ia.album_id
GROUP BY invoice_id, ia.album_id
), final_count AS
(
SELECT *, 
CASE
WHEN tracks_invoice = tracks_album
THEN 1
ELSE 0
END AS type_invoice
FROM tracks_count
GROUP BY invoice_id
HAVING tracks_album > 2
)

Select MAX(invoice_id) Total_invoices, CAST(SUM(type_invoice) AS FLOAT)/614  from final_count;

Done.


Total_invoices,CAST(SUM(type_invoice) AS FLOAT)/614
614,0.18078175895765472


Around 18% of the invoices are album purchases. Based on our findings, we recomment Chinook Company to avoid full album purchases and purchase only tracks from Albums.